# Acceleration of the cell cycle drives the outgrowth of the regenerating spinal cord in axolotls

Fabian Rost<sup>1†</sup>, Aida Rodrigo Albors<sup>*2,3,4†</sup>, Vladimir Mazurov<sup>3,4</sup>, Lutz Brusch<sup>1</sup>, Andreas Deutsch<sup>1</sup>, Elly M. Tanaka<sup>3,4‡*</sup> and Osvaldo Chara<sup>1,5‡*</sup>

<sup>1</sup> Center for Information Services and High Performance Computing, Technische Universität Dresden, Dresden, Germany<br>
<sup>2</sup>Division of Cell & Developmental Biology, School of Life Sciences, University of Dundee, Dundee, United Kingdom<br>
<sup>3</sup> Deutsche Forschungsgemeinschaft – Center for Regenerative Therapies Dresden, Dresden, Germany<br>
<sup>4</sup> Max Planck Institute of Molecular Cell Biology and Genetics, Dresden, Germany<br>
<sup>5</sup> Systems Biology Group (SysBio), Institute of Physics of Liquids and Biological Systems, National Scientific and Technical Research Council (CONICET) and University of La Plata, La Plata, Argentina<br>

<sup>†</sup> Co-first authorship
<sup>‡</sup> Co-last authorship
<sup>\*</sup> Corresponding authors

Corresponding authors:

Osvaldo Chara<br>
Center for Information Services and High Performance Computing, Technische Universität Dresden,
Nöthnitzer Straße 46, 01187 Dresden, Germany<br>
Tel. +49 351 463-38780<br>
E-mail: osvaldo.chara@tu-dresden.de<br>

Systems Biology Group (SysBio), Institute of Physics of Liquids and Biological Systems (IFLySIB), National Scientific and Technical Research Council (CONICET) and University of La Plata<br>
Calle 59 N 789, 1900 La Plata, Argentina<br>
Tel. +54 221 4233283 Ext: 26<br>
E-mail: ochara@iflysib.unlp.edu.ar

Elly Tanaka<br>
CRTD<br>
Fetscherstraβe 105, 01307 Dresden, Germany<br>
Tel. +49 351 458-82000<br>
Fax +49 351 458-82009<br>
E-mail: elly.tanaka@crt-dresden.de

## Introduction
These notebook were used to perform data analysis presented in Rost et al., in preparation. It contains all the data and the code for the data analysis. 
The code is only partly documented. If you have questions concerning the code, please contact the authors.

Please also make sure you visited https://github.com/fabianrost84/Rost-Rodrigo-Albors-et-al-2016 to make sure you have the latest version of the notebooks and for getting instructions how to view them and/or get them running.

## Dependencies
The notebooks depend on the python modules below. Python 2.7 was used. If the import works, the notebooks should hopefully work. The repository also contains an `environment.yml` file which can be used with conda to set up a working python environment.

In [1]:
%load_ext ipycache
import cloudpickle
import collections
import datetime
import functools
import gc
import iminuit
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import ticker
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import os
import multiprocessing
import numba
import probfit
import pymc
import tables
import random
import scipy as sp
from scipy import stats
import scipy.optimize
import shutil
from uncertainties import ufloat
import warnings

/home/fabian/anaconda2/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/fabian/anaconda2/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


## List of notebooks

* [Outgrowth](calculations/outgrowth.ipynb):
    * caclualtion of the mean outgrowth dynamics for Fig. 1B
* [Constant density](calculations/spatial_analysis/constant_density.ipynb)
    * performing tests described in Materials and methods section "Testing for spatial dependence of SOX2+ cell counts"
    * Fig. 2B,B',C
* [simulation_spatial_model](calculations/spatial_analysis/simulation_spatial_model.ipynb):
    * Fig. 2 - figure supplement 2
* [spatial_model_sketch](calculations/spatial_analysis/spatial_model_sketch.ipynb):
    * Fig. 2E'
* [step_model_fixed_density_fit_per_timepoint](calculations/spatial_analysis/step_model_fixed_density_fit_per_timepoint.ipynb):
    * fitting the model of two zones of proliferation to the cell count data 
* [step_model_fixed_density_visualize_fit_results_per_timepoint_constant_density](calculations/spatial_analysis/step_model_fixed_density_visualize_fit_results_per_timepoint_constant_density.ipynb):
    * visualization of the above fit results
    * Fig. 2D,D',F-F'', Figure 2 - figure supplement 1, Figure 2 - figure supplement 3, Figure 2 - figure supplement 4
* [step_model_fixed_density_fit_per_timepoint_for_hpc_1e7.py](calculations/spatial_analysis/step_model_fixed_density_fit_per_timepoint_for_hpc_1e7.py):
    * python file used to run the spatial model fit on the TU Dresden HPC
* [cell_cycle_length_kinetics](calculations/cell_cycle_length_kinetics.ipynb):
    * calculation of proliferation rate time course in high-proliferation zone
    * Fig. 2G, Figure 2 - figure supplement 5
* [number_quiescent_cells](calculations/number_quiescent_cells.ipynb):
    * calculation of time course of the total number of SOX2<sup>+</sup>/PCNA<sup>-</sup> cells in the high proliferation zone
    * Fig. 2H
* [clone_velocities](calculations/clone_velocities/clone_velocities.ipynb):
    * calculation of the clone velocities from the trajectories
    * Fig. 2J,K
* [brdu_bootstrapping_day6](calculations/brdu_bootstrapping_day6.ipynb):
    * estimating the cell cycle length at day 6 using bootstrapping with a case resampling scheme. Data and model used for this were described in Rodrigo Albors et al., 2015
* [lg_model](calculations/lg_model/lg_model.ipynb):
    * numerical implementation of the mechanistic model of spinal cord outgrowth (Equations (1) & (2))
* [plot_lg_model_results](calculations/lg_model/plot_lg_model_results.ipynb):    
    * Fig. 3B-G, Fig. 3 - figure supplement 1

# References

Rodrigo Albors, A., Tazaki, A., Rost, F., Nowoshilow, S., Chara, O., & Tanaka, E. M. (2015). *Planar cell polarity-mediated induction of neural stem cell expansion during axolotl spinal cord regeneration.* eLife, e10230. http://doi.org/10.7554/eLife.10230